In [7]:
from sqlalchemy import create_engine
import pandas as pd

In [4]:
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
.format(username=POSTGRES_USERNAME,
password=POSTGRES_PASSWORD,
ipaddress=POSTGRES_ADDRESS,
port=POSTGRES_PORT,
dbname=POSTGRES_DBNAME))
cnx = create_engine(postgres_str, connect_args={'sslmode':'require'})

In [5]:
q = """
SELECT 
        DISTINCT ON(st_id) st_id as active_users_id
FROM 
        peas
WHERE 
        date_trunc('month', timest) = '2020-03-01' and correct = True
GROUP BY
        st_id
    ,   date_trunc('hour', timest)
HAVING
count(correct) >= 20
LIMIT 5
"""

In [6]:
pd.read_sql(q, cnx)

,active_users_id
0,1
1,6
2,9
3,12
4,18


In [32]:
q = """
WITH client AS (SELECT 
      DISTINCT ON(r.st_id) r.st_id AS client_id
FROM studs AS l RIGHT JOIN checks AS r
ON l.st_id = r.st_id)
,   active_user AS (SELECT 
        DISTINCT ON(st_id) st_id AS active_user_id
     FROM peas WHERE correct = True GROUP BY
         st_id
    ,    date_trunc('hour', timest)
     HAVING count(correct) >= 20)
,   active_math_user AS (SELECT
    DISTINCT ON(st_id) st_id AS active_math_user_id
FROM peas WHERE subject = 'math' and correct = True GROUP BY
       st_id
    ,  date_trunc('hour', timest)
HAVING count(correct) >= 20)
,
 active_client AS (SELECT
        r.st_id AS active_client_id FROM client AS l
LEFT JOIN peas AS r
ON l.client_id = r.st_id
WHERE correct = True
GROUP BY
        r.st_id
    ,   date_trunc('hour', timest)
HAVING count(correct) >= 20)
,
    active_math AS (SELECT
        r.st_id AS active_math_id
FROM
        client AS l
LEFT JOIN peas AS r ON l.client_id = r.st_id
WHERE subject = 'math' and correct = True
GROUP BY
        r.st_id
    ,   date_trunc('hour', timest)
HAVING count(correct) >= 20)
SELECT 
        SUM(money) / (SELECT COUNT(st_id) FROM studs) AS ARPU
    ,   SUM(money) / (SELECT COUNT(active_user_id) FROM active_user) AS ARPAU
    ,   (SELECT count(client_id)::numeric FROM client) /
    (SELECT COUNT(st_id) FROM studs) AS CR
    ,   (SELECT count(active_client_id)::numeric FROM active_client) /
    (SELECT count(active_user_id)::numeric FROM active_user) 
    AS active_CR
    ,   (SELECT count(active_math_id)::numeric FROM active_math) / 
    (SELECT count(active_math_user_id) FROM active_math_user)
    AS active_math_CR
    FROM checks

"""

In [33]:
pd.read_sql(q, cnx)

,arpu,arpau,cr,active_cr,active_math_cr
0,399.71,761.352381,0.25,0.238095,0.2
